<a href="https://colab.research.google.com/github/dgg32/crop_area/blob/main/Google_Earth_Engine_USA_Crops_Area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run me first

First of all, run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [12]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=fNmvQhVY6QDM4q-TTeLRegCfZoPdQZjkBYoQlcY3Oe4&tc=odzzIRTPDY5XiPs096rXYo0R24Vs7B94VMjuP0TmqRs&cc=8HCwqpaT474MP48U6MvMdRZ-3BMALkeXjYgGxHPM40g

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqN-RxtC25fimK0WakJXvYNTnb2MsENH8DKPvLKdtR2U5bPrLfJ1LA

Successfully saved authorization token.


In [34]:
startDate = '2015-01-01'
endDate = '2016-01-01'

areaPerPixel = ee.Image.pixelArea()

dataset = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(startDate, endDate)).select("cropland").first()
provinces = ee.FeatureCollection("FAO/GAUL/2015/level2");

targets = ["Corn", "Cotton", "Rice"]
target_index = 0

state_list = ee.List(["Alabama", "Colorado"]);

def calculate_area (state):
    def generate_collection(feature):
      geometry = feature.geometry();
      
                  
      o_dataset = dataset.clip(geometry);


      image = dataset.eq(target_index).selfMask().multiply(areaPerPixel).divide(1e6).set('system:time_start', startDate);
      
      
      reduce = image.select('cropland').reduceRegion(
                                                reducer= ee.Reducer.sum(),
                                                geometry= geometry,
                                                scale= 30,
                                                bestEffort= True
                                            );
            
      return feature.set(target, reduce.get("cropland"))

    filterLevel1 = ee.Filter.eq('ADM1_NAME', state);
    o_state = provinces.filter(filterLevel1);

    corn_areas = o_state.map(generate_collection)

    results = corn_areas.reduceColumns(ee.Reducer.sum(), [target])


    return results.values(["sum"]).get(0)

tasks = []

for target in targets:
    #target = "Corn";
    target_index = dataset.get("cropland_class_names").getInfo().index(target);

    results = state_list.map(calculate_area).reduce(ee.Reducer.sum());

    

    #print (target, results.getInfo())

    task = ee.batch.Export.table.toDrive(collection=ee.FeatureCollection([ee.Feature(None, {"time": startDate, target: results})]),
                                     description=f'{target}_python_crop',
                                     fileFormat='CSV')
    task.start()

    tasks.append(task)

In [36]:
for task in tasks:
    print (task.status())

{'state': 'COMPLETED', 'description': 'Corn_python_crop', 'creation_timestamp_ms': 1664181341704, 'update_timestamp_ms': 1664181351400, 'start_timestamp_ms': 1664181347861, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'batch_eecu_usage_seconds': 0.9888148903846741, 'id': 'TPVUD66NCNMDKL7HNSMQYPJZ', 'name': 'projects/earthengine-legacy/operations/TPVUD66NCNMDKL7HNSMQYPJZ'}
{'state': 'COMPLETED', 'description': 'Cotton_python_crop', 'creation_timestamp_ms': 1664181342139, 'update_timestamp_ms': 1664181354458, 'start_timestamp_ms': 1664181348736, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'batch_eecu_usage_seconds': 1.4602303504943848, 'id': '6H4B3YDIUIZQF2CGLVI6SDXU', 'name': 'projects/earthengine-legacy/operations/6H4B3YDIUIZQF2CGLVI6SDXU'}
{'state': 'COMPLETED', 'description': 'Rice_python_crop', 'creation_timestamp_ms': 1664181342571, 'update_timestamp_ms': 1664181357586, 'start_t